In [ ]:
import os
os.chdir('..')

from lucifex.sim import Simulation, integrate
from lucifex.viz import plot_line, plot_colormap
from lucifex.utils import where_on_grid

from simulate import carbon_dissolution
from postprocess_plots_many_to_one import plot_colormap_comparison


u_options = {'MFEM': ..., 'streamfunction': (..., None)}
simulation: dict[str, Simulation] = {}
sim_func = carbon_dissolution(store_step=0.01)

for k, v in u_options.items():
    sim = sim_func(u_options=v, cell='left_right', Ra=600.0, Da=100.0, Nx=80, Ny=80, c_bounds=(True, True))
    integrate(sim_func, n_stop=200, dt_init=1e-6, n_init=5)
    simulation[k] = sim

In [ ]:
plot_colormap_comparison(
    [i['c'] for i in simulation.values()],
    t=-1,
    labels=[f"method='{c}'" for c in u_options],
)

[5.413663774090455,
 5.622032739329121,
 5.771757901310468,
 5.898361315825434,
 6.016396531802979,
 6.133391208891468,
 6.253448143847867,
 6.379362535880452,
 6.514146143613648,
 6.662379697120562]

In [ ]:
umax = simulation['umax']
plot_line((umax.time_series, umax.value_series))

In [ ]:
cminmax = simulation['cminmax']
plot_line((cminmax.time_series, [i[0] for i in cminmax.value_series]))
plot_line((cminmax.time_series, [i[1] for i in cminmax.value_series]))

In [ ]:
dt = simulation['dt']
c = simulation['c']
umax = simulation['umax']
cminmax = simulation['cminmax']
Sminmax = simulation['Sminmax']

plot_colormap(simulation['c'][-1])
plot_line((umax.time_series, [i for i in umax.value_series]))
plot_line((cminmax.time_series, [i[1] for i in cminmax.value_series]))
plot_line((cminmax.time_series, [i[0] for i in cminmax.value_series]))
plot_line((Sminmax.time_series, [i[0] for i in Sminmax.value_series]))

In [ ]:
plot_line((cminmax.time_series[:50], [i[1] for i in cminmax.value_series][:50]))

In [ ]:
[i[1] for i in cminmax.value_series]

In [ ]:
from lucifex.sim import integrate
from lucifex.viz import plot_line, plot_colormap
from simulate import carbon_dissolution


RA = 5000
DA = 100
# NCELL = (100, 150, 200)
NCELL = (150,)
SUPG = (None, 'codina', 'shakib')
# NX_NY = {i for j in [((n, n), (min(NCELL), n), (n, min(NCELL))) for n in NCELL] for i in j}

T_STOP = 10.0
N_STOP = None
DT_INIT = 1E-6
N_INIT = 15

for supg in (None, 'codina', 'shakib'):
    for n in NCELL:
        simulation = carbon_dissolution(
                            store_step=1, 
                            write_step=None, 
                            file_name=('FunctionSeries', 'ConstantSeries'),
                            dir_base='./data/mesh_resolution',
                            dir_labels=('Nx', 'Nx', 'supg'),
                            dir_timestamp=True,
                        )(Ra=RA, Da=DA, Nx=n, Ny=n, supg=supg)
        integrate(simulation, t_stop=T_STOP, n_stop=N_STOP, dt_init=DT_INIT, n_init=N_INIT)
        plot_colormap(simulation['c'][-1])

In [ ]:
from lucifex.fdm import AB, CN
from lucifex.sim import integrate
from simulate import carbon_dissolution


RA = 5000
DA = 1000
NCELL = (100, 150, 200)
SUPG = (None, 'codina', 'shakib')
NX_NY = {i for j in [((n, n), (min(NCELL), n), (n, min(NCELL))) for n in NCELL] for i in j}

T_STOP = 12.0
N_STOP = None
DT_INIT = 1E-6
N_INIT = 15

for supg in (None, 'codina', 'shakib'):
    for (nx, ny) in NX_NY:
        simulation = carbon_dissolution(
                            store_step=None, 
                            write_step=1, 
                            file_name=('FunctionSeries', 'ConstantSeries'),
                            dir_base='./data/mesh_resolution',
                            dir_labels=('Nx', 'Nx', 'supg'),
                            dir_timestamp=True,
                        )(Ra=RA, Da=DA, Nx=nx, Ny=ny, supg=supg)
        integrate(simulation, t_stop=T_STOP, n_stop=N_STOP, dt_init=DT_INIT, n_init=N_INIT)